# 🧪 Statystyka i Skalowanie Danych

W tym notatniku przećwiczymy dwa kluczowe pojęcia w Data Science:

1.  **Hipoteza Zerowa i P-value:** Jak matematycznie sprawdzić, czy nasz eksperyment (np. nowy lek) naprawdę działa, czy to tylko przypadek.
2.  **Skalowanie (Standaryzacja):** Jak przygotować dane liczbowe (np. Wiek i Zarobki), aby algorytmy Machine Learning nie "zgłupiały" przez różnice w wielkości liczb.

In [7]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler

print("Biblioteki załadowane! Możemy zaczynać.")

Biblioteki załadowane! Możemy zaczynać.


## CZĘŚĆ 1: P-value i Hipoteza Zerowa (Symulacja Medyczna)

Wyobraź sobie, że testujemy nowy lek na odchudzanie.
Mamy dwie grupy po 100 osób:
1.  **Grupa A (Placebo):** Dostaje zwykły cukierek.
2.  **Grupa B (Lek):** Dostaje naszą nową substancję.

### Zasady gry:
*   **Hipoteza Zerowa ($H_0$):** "Nuda. Lek nie działa. Różnice w wadze to czysty przypadek."
*   **P-value:** Liczba, która mówi: "Jaka jest szansa, że ten wynik to przypadek?".
    *   Jeśli **p < 0.05** (mniej niż 5%) -> To nie przypadek! **Lek działa.** (Odrzucamy $H_0$)
    *   Jeśli **p > 0.05** -> To może być przypadek. Nie mamy dowodu. (Zostajemy przy $H_0$)

In [8]:
# Ustawiamy "ziarno" losowości, żeby wyniki były powtarzalne
np.random.seed(42)

# 1. GENERUJEMY DANE (Symulacja)
# Grupa Placebo: Średnio chudną 2 kg, odchylenie standardowe 1.5
placebo = np.random.normal(loc=2.0, scale=1.5, size=100)

# Grupa Leku: Średnio chudną 3 kg (LEPIEJ!), odchylenie standardowe 1.5
lek = np.random.normal(loc=3.0, scale=1.5, size=100)

print(f"Średnia utrata wagi (Placebo): {np.mean(placebo):.2f} kg")
print(f"Średnia utrata wagi (Lek):     {np.mean(lek):.2f} kg")
print("-" * 50)

# 2. TEST STATYSTYCZNY (T-test)
# Sprawdzamy matematycznie, czy ta różnica (2kg vs 3kg) jest istotna.
t_stat, p_value = stats.ttest_ind(placebo, lek)

print(f"P-value wynosi: {p_value:.10f}") 
# .10f oznacza 10 miejsc po przecinku

print("-" * 50)
print("WERDYKT:")

# 3. INTERPRETACJA AUTOMATYCZNA
if p_value < 0.05:
    print("✅ P jest bliskie zera (mniejsze niż 0.05).")
    print("To na pewno NIE jest przypadek.")
    print("WNIOSEK: Odrzucamy hipotezę zerową -> LEK DZIAŁA!")
else:
    print("❌ P jest duże (większe niż 0.05).")
    print("To mógł być przypadek.")
    print("WNIOSEK: Nie mamy dowodów, że lek działa.")

Średnia utrata wagi (Placebo): 1.84 kg
Średnia utrata wagi (Lek):     3.03 kg
--------------------------------------------------
P-value wynosi: 0.0000000083
--------------------------------------------------
WERDYKT:
✅ P jest bliskie zera (mniejsze niż 0.05).
To na pewno NIE jest przypadek.
WNIOSEK: Odrzucamy hipotezę zerową -> LEK DZIAŁA!


## CZĘŚĆ 2: Skalowanie Danych (MinMax vs Standard)

Algorytmy (np. KNN, Sieci Neuronowe) widzą świat przez pryzmat liczb.
Jeśli masz:
*   **Wiek:** 30
*   **Zarobki:** 10 000

Dla algorytmu zmiana zarobków o 100 jednostek jest "większa i ważniejsza" niż zmiana wieku o 10 lat. **To błąd!** Musimy sprowadzić te dane do "wspólnego mianownika".

Mamy dwie główne metody:
1.  **Min-Max Scaler:** Ściska dane do przedziału **[0, 1]**. (Dobre dla obrazów).
2.  **Standard Scaler:** Przesuwa średnią do **0**. (Dobre dla większości algorytmów ML).

In [9]:
# 1. Tworzymy dane z ogromną różnicą skali
df = pd.DataFrame({
    'Wiek': [20, 30, 40, 50, 60],        # Małe liczby
    'Zarobki': [3000, 4500, 8000, 12000, 20000] # Duże liczby
})

print("--- DANE ORYGINALNE ---")
print(df)
print("\n")

# --- METODA A: MIN-MAX SCALER ---
# Wszystko będzie pomiędzy 0 a 1
scaler_minmax = MinMaxScaler()
df_minmax = pd.DataFrame(scaler_minmax.fit_transform(df), columns=['Wiek', 'Zarobki'])

print("--- PO MIN-MAX SCALER (Wartości 0-1) ---")
print(df_minmax)
print("\n")

# --- METODA B: STANDARD SCALER ---
# Średnia = 0. Wartości ujemne = poniżej średniej.
scaler_std = StandardScaler()
df_std = pd.DataFrame(scaler_std.fit_transform(df), columns=['Wiek', 'Zarobki'])

print("--- PO STANDARD SCALER (Wokół zera) ---")
print(df_std.round(2))

--- DANE ORYGINALNE ---
   Wiek  Zarobki
0    20     3000
1    30     4500
2    40     8000
3    50    12000
4    60    20000


--- PO MIN-MAX SCALER (Wartości 0-1) ---
   Wiek   Zarobki
0  0.00  0.000000
1  0.25  0.088235
2  0.50  0.294118
3  0.75  0.529412
4  1.00  1.000000


--- PO STANDARD SCALER (Wokół zera) ---
   Wiek  Zarobki
0 -1.41    -1.07
1 -0.71    -0.82
2  0.00    -0.25
3  0.71     0.41
4  1.41     1.72
